# Create Edge .CSV's per USA County

In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd

#pd.set_option('display.max_rows', 3500)


### Read in county names from 2010 USA census data (https://www.census.gov/geo/reference/codes/cou.html).

In [ ]:
counties_raw = pd.read_table('../inputData/national_counties.txt',header=None)
counties_raw.rename(columns={0:'State, County'})
tmp0 = []
tmp1 = []
for i in range(0,len(counties_raw)):
    tmp0.append(counties_raw.iloc[i,0][10:-3])
    tmp1.append(counties_raw.iloc[i,0][:2])
counties = pd.DataFrame.from_dict(tmp0)
states = pd.DataFrame.from_dict(tmp1)

counties.rename(columns={0:'County'})
states.rename(columns={0:'State'})

counties = counties[states!='AS'].dropna()
counties = counties[states!='AK'].dropna()
counties = counties[states!='GU'].dropna()
counties = counties[states!='MP'].dropna()
counties = counties[states!='PR'].dropna()
counties = counties[states!='VI'].dropna()
counties = counties[states!='UM'].dropna()
counties = counties.reset_index()

#Baltimore needs city --> county:
counties[0][1187] = counties[0][1187].replace('city','County')
counties[0][1568] = counties[0][1568].replace(' city','')
#Charlotsville city needs --> Charlostville
counties[0][2890] = counties[0][2890].replace(' city','')
counties[0][2896] = counties[0][2896].replace('city','County')
counties[0][2902] = counties[0][2902].replace(' city','')
counties[0][2905] = counties[0][2905].replace(' city','')
counties[0][2906] = counties[0][2906].replace(' city','')
counties[0][2914] = counties[0][2914].replace(' city','')
counties[0][2917] = counties[0][2917].replace(' city','')
counties[0][2920] = counties[0][2920].replace(' city','')
counties[0][2921] = counties[0][2921].replace(' city','')
counties[0][2922] = counties[0][2922].replace(' city','')
counties[0][2923] = counties[0][2923].replace('city','County')



#MISSING BEAUFORT COUNTY, NC
#MISSING ACCOMACK COUNTY, VA
#INCOMPLETE BEAUFORT COUNTY, SC


states = states[states!='AS'].dropna()
states = states[states!='AK'].dropna()
states = states[states!='GU'].dropna()
states = states[states!='MP'].dropna()
states = states[states!='PR'].dropna()
states = states[states!='VI'].dropna()
states = states[states!='UM'].dropna()
states = states.reset_index()

#Hennepin County needs MN --> Minnesota:
states[0][1311] = states[0][1311].replace('MN','Minnesota')


counties
#states

### Loop through counties to get OSMnx graphs, get edges and save to .csv

In [ ]:
for i in range(2923, len(counties)):
    # name of county:
    place_name = counties[0][i] + ', ' + states[0][i]
    # map network:
    print('obtaining graph for '+place_name+'...')
    graph = ox.graph_from_place(place_name, network_type='drive')
    # street network:
    edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)
    # write path:
    path = '/Users/samtaimourzadeh/python/dev/osm/edges/' + states[0][i]+'_'+counties[0][i]+'.csv'
    path = path.replace(' ','')
    # write to csv:
    edges.to_csv(path)
    print('saved ' + path)
    print(i/len(counties)*100, ' percent complete.')

## Explode edge .csv's into edge_exploded directory

In [ ]:
import os

In [6]:
edges_csv = os.listdir('../edges')
len(edges_csv)
edges_csv[0]

'KS_GreeleyCounty.csv'

In [ ]:
for n in range(0, len(edges_csv)):
    edges = pd.read_csv('../edges/'+edges_csv[n], low_memory=False)
    # number of edges
    nedges = edges.geometry.size
    # get longitude/latitude dataFrame for edges
    df_edge_loc = pd.DataFrame()
    print('Creating exploded dataframe for '+edges_csv[n].replace('.csv',''))
    for i in range(0, nedges):
        edge_string = edges['geometry'][i][12:-1].split(",")
        tmp = []
        for j in range(0, len(edge_string)):
            edge_array = np.fromstring(edge_string[j], sep=' ')
            edge_array = np.flip(edge_array)
            tmp.append(edge_array)
            tmp2 = np.concatenate(tmp)
            dftmp = pd.DataFrame(tmp2)
        df_edge_loc = pd.concat([df_edge_loc, dftmp], axis=1, ignore_index=True)
    path = '../edges_exploded/'+edges_csv[n].replace('.csv','_exploded.csv')
    df_edge_loc.to_csv(path)
    del df_edge_loc
    print('Saved '+path)
    print(100*n/len(edges_csv),' percent complete.')

In [ ]:
#MO_HoltCounty Error: /anaconda3/envs/osm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785:
#DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=
#False.interactivity=interactivity, compiler=compiler, result=result)